In [1]:
import gc
import math
import random
import numpy as np
import copy
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, SimpleRNN, GRU, LSTM, Dropout, TimeDistributed, Input
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import  SGD, RMSprop, Adam
from functools import cmp_to_key
from google.colab import drive
drive.mount('/content/gdrive')
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/10_nodes_new')

edges = [[1, 2], [1, 5], [1, 8], [2, 3], [3, 4], [3, 8], [3, 9], [4, 10], [5, 6], [6, 7], [6, 8], [6, 9], [7, 10], [9, 10]]
number_edges = len(edges)
number_steps = 4
start_node = 1
end_node = 10

df = pd.read_csv('gdrive/My Drive/Colab Notebooks/10_nodes_new/data/10_nodes_LU.csv')
x = df.iloc[:,:number_edges].values
y = df.iloc[:,number_edges:].values

df_hp = pd.read_csv('gdrive/My Drive/Colab Notebooks/10_nodes_new/data/10_nodes_LU_hp.csv')
x_hp = df_hp.iloc[:,:number_edges].values
y_hp = df_hp.iloc[:,number_edges:].values

from sklearn.model_selection import train_test_split
x_main_train,x_main_tv,y_main_train,y_main_tv = train_test_split(x,y,test_size=2/5)
x_main_test,x_main_valid,y_main_test,y_main_valid = train_test_split(x_main_tv,y_main_tv,test_size=1/2)

x_hp_train,x_hp_tv,y_hp_train,y_hp_tv = train_test_split(x_hp,y_hp,test_size=2/5)
x_hp_test,x_hp_valid,y_hp_test,y_hp_valid = train_test_split(x_hp_tv,y_hp_tv,test_size=1/2)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_main_train)
x_test = sc.transform(x_main_test)
x_valid = sc.transform(x_main_valid)

sc_hp = StandardScaler()
x_train_1 = sc_hp.fit_transform(x_hp_train)
x_test_1 = sc_hp.transform(x_hp_test)
x_valid_1 = sc_hp.transform(x_hp_valid)

x_train = np.repeat(x_train[:, np.newaxis, : ], number_steps, axis=1)
x_test = np.repeat(x_test[:, np.newaxis, : ], number_steps, axis=1)
x_valid = np.repeat(x_valid[:, np.newaxis, : ], number_steps, axis=1)

x_train_1 = np.repeat(x_train_1[:, np.newaxis, : ], number_steps, axis=1)
x_test_1 = np.repeat(x_test_1[:, np.newaxis, : ], number_steps, axis=1)
x_valid_1 = np.repeat(x_valid_1[:, np.newaxis, : ], number_steps, axis=1)

y_train = np.reshape(y_main_train,(y_main_train.shape[0], number_steps, number_edges))
y_test = np.reshape(y_main_test,(y_main_test.shape[0], number_steps, number_edges))
y_valid = np.reshape(y_main_valid,(y_main_valid.shape[0], number_steps, number_edges))

y_train_1 = np.reshape(y_hp_train,(y_hp_train.shape[0], number_steps, number_edges))
y_test_1 = np.reshape(y_hp_test,(y_hp_test.shape[0], number_steps, number_edges))
y_valid_1 = np.reshape(y_hp_valid,(y_hp_valid.shape[0], number_steps, number_edges))

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape, x_valid.shape, y_valid.shape, x_train_1.shape, y_train_1.shape, x_test_1.shape, y_test_1.shape)

# Hyperparameters of structure: number of layers, number of units, dropout, cell type
# Hyperparameters of training: learning rate, batch size
HP_1 = [[1, 2, 3, 4, 5],
       [-1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, -1],
       [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
       [SimpleRNN, GRU, LSTM],
       [SGD, RMSprop, Adam],
       [0.0001, 0.001, 0.01, 0.1],
       [16, 32, 64, 128, 256, 512, 1024]]

t = []
for p in HP_1:
    t.append(len(p))
t1 = []
hp = []
for i in range(t[0]):
    t1.append(t[1])
    t1.append(t[2])
    hp.append(HP_1[1])
    hp.append(HP_1[2])
for i in range(3, len(HP_1)):
    t1.append(t[i])
    hp.append(HP_1[i])
t2 = [int(np.sqrt(itm)) for itm in t1]
n_hp = 2*t[0] + len(HP_1) - 3
print(t, t1, t2, n_hp)

from Results import Results
def custom_accuracy(y_true, y_pred):
    threshold = 0.5
    y_pred_binary = tf.cast(tf.greater_equal(y_pred, threshold), tf.float32)
    y_true = tf.cast(y_true, tf.float32)
    stepwise_correct = tf.reduce_all(tf.equal(y_true, y_pred_binary), axis=-1)
    sequencewise_correct = tf.reduce_all(stepwise_correct, axis=-1)
    acc = tf.reduce_mean(tf.cast(sequencewise_correct, tf.float32))
    return acc

callback = tf.keras.callbacks.EarlyStopping(monitor='val_custom_accuracy',
                                            min_delta=0.001, patience=5, verbose=1, mode='max',
                                            restore_best_weights=True)

Mounted at /content/gdrive
(600000, 4, 14) (600000, 4, 14) (200000, 4, 14) (200000, 4, 14) (200000, 4, 14) (200000, 4, 14) (6000, 4, 14) (6000, 4, 14) (2000, 4, 14) (2000, 4, 14)
[5, 22, 6, 3, 3, 4, 7] [22, 6, 22, 6, 22, 6, 22, 6, 22, 6, 3, 3, 4, 7] [4, 2, 4, 2, 4, 2, 4, 2, 4, 2, 1, 1, 2, 2] 14


In [2]:
best_hps = []
with open('gdrive/My Drive/Colab Notebooks/10_nodes_new/RS/acc/RS.txt') as f:
    lines = f.readlines()
    for line in lines:
        a = line.split(",")
        b = [int(i) for i in a]
        best_hps.append(b)
best_hps

[[14, 2, 12, 5, 12, 1, 1, 1, 18, 5, 1, 1, 2, 2],
 [20, 1, 5, 0, 10, 2, 16, 3, 15, 0, 1, 2, 2, 4],
 [1, 1, 3, 2, 10, 3, 2, 1, 3, 2, 0, 1, 3, 1],
 [1, 1, 21, 1, 1, 3, 6, 0, 1, 0, 1, 1, 2, 6],
 [21, 0, 0, 4, 0, 3, 5, 2, 0, 3, 0, 0, 3, 0],
 [8, 4, 0, 1, 21, 2, 4, 4, 5, 4, 0, 0, 1, 2],
 [1, 0, 8, 0, 2, 4, 1, 4, 2, 5, 1, 0, 3, 2],
 [13, 2, 13, 2, 8, 4, 4, 2, 17, 3, 2, 2, 2, 1],
 [8, 5, 21, 1, 1, 3, 6, 0, 0, 1, 0, 1, 1, 3],
 [21, 3, 16, 4, 21, 3, 3, 0, 11, 2, 1, 2, 2, 4],
 [21, 0, 0, 1, 2, 0, 4, 5, 3, 4, 1, 2, 0, 6],
 [2, 4, 6, 5, 8, 1, 21, 4, 1, 5, 0, 1, 1, 6],
 [17, 5, 16, 4, 2, 3, 12, 5, 13, 4, 2, 2, 1, 1],
 [14, 3, 7, 2, 6, 3, 12, 5, 21, 0, 2, 2, 1, 1],
 [14, 5, 16, 4, 9, 3, 16, 3, 0, 0, 2, 1, 1, 0],
 [7, 4, 18, 3, 16, 4, 13, 5, 10, 2, 1, 2, 1, 0],
 [5, 3, 20, 1, 21, 5, 21, 4, 18, 2, 2, 1, 1, 0],
 [7, 2, 21, 3, 12, 2, 15, 5, 19, 5, 2, 1, 2, 2],
 [19, 4, 12, 2, 11, 2, 21, 2, 16, 2, 0, 1, 2, 4],
 [21, 5, 2, 5, 21, 5, 0, 3, 0, 5, 1, 1, 0, 4],
 [17, 1, 14, 3, 9, 0, 21, 5, 2, 1, 1, 2, 2, 5],
 

In [3]:
data_x_train = x_train
data_y_train = y_train

data_x_valid = x_valid
data_y_valid = y_valid

data_x_test = x_test
data_y_test = y_test

data_x_original_train = x_main_train
data_x_original_test = x_main_test

In [4]:
# data_x_train = x_train_1
# data_y_train = y_train_1

# data_x_valid = x_valid_1
# data_y_valid = y_valid_1

# data_x_test = x_test_1
# data_y_test = y_test_1

# data_x_original_train = x_hp_train
# data_x_original_test = x_hp_test

In [5]:
for m in range(20, 30):
    tf.keras.backend.clear_session()
    input_shape = (number_steps, number_edges)
    inputs = Input(shape=input_shape)
    x_input = inputs
    cell_type = hp[2*t[0]][best_hps[m][2*t[0]]]
    for i in range(t[0]):
        if hp[2*i][best_hps[m][2*i]] != -1:
            x_input = cell_type(hp[2*i][best_hps[m][2*i]], return_sequences=True)(x_input)
            x_input = Dropout(hp[2*i+1][best_hps[m][2*i+1]])(x_input)
    outputs = TimeDistributed(Dense(number_edges, activation="sigmoid"))(x_input)
    model = Model(inputs=inputs, outputs=outputs)
    opt = hp[2*t[0]+1][best_hps[m][2*t[0]+1]]
    model.compile(loss=BinaryCrossentropy(), optimizer=opt(learning_rate=hp[2*t[0]+2][best_hps[m][2*t[0]+2]]), metrics = [custom_accuracy])
    model.summary()
    history = model.fit(data_x_train, data_y_train, epochs=1000, batch_size=hp[2*t[0]+3][best_hps[m][2*t[0]+3]], callbacks=[callback], validation_data=(data_x_valid, data_y_valid), verbose=1)

    r = Results(number_edges, edges, start_node, end_node, number_steps)
    pred = model.predict(data_x_train, verbose = 0)
    pred_test = model.predict(data_x_test, verbose = 0)
    accuracy_train = list(r.get_accuracy(pred, data_x_original_train, data_y_train))
    accuracy_test = list(r.get_accuracy(pred_test, data_x_original_test, data_y_test))

    with open("/content/gdrive/My Drive/Colab Notebooks/10_nodes_new/RS/acc/history_model/history_{}.txt".format(m+1), "w") as output:
        output.write(str(history.history['custom_accuracy'])+'\n')
        output.write(str(history.history['val_custom_accuracy'])+'\n')
        output.write(str(accuracy_train)+'\n')
        output.write(str(accuracy_test)+'\n')

    model.save('gdrive/My Drive/Colab Notebooks/10_nodes_new/RS/acc/history_model/model_{}.h5'.format(m+1))

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 4, 14)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ (None, 4, 170)              │          94,860 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 4, 170)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 4, 140)              │         131,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 4, 140)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_2 (GRU)                          │ (None, 4, 90)               │          62,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 4, 90)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_3 (GRU)                          │ (None, 4, 20)               │           6,720 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 4, 20)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ (None, 4, 14)               │             294 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 295,554 (1.13 MB)

 Trainable params: 295,554 (1.13 MB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 4, 14)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ (None, 4, 30)               │           4,140 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 4, 30)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 4, 180)              │         114,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 4, 180)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_2 (GRU)                          │ (None, 4, 150)              │         149,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 4, 150)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_3 (GRU)                          │ (None, 4, 150)              │         135,900 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 4, 150)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_4 (GRU)                          │ (None, 4, 140)              │         122,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 4, 140)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ (None, 4, 14)               │           1,974 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 528,534 (2.02 MB)

 Trainable params: 528,534 (2.02 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/1000
4688/4688 ━━━━━━━━━━━━━━━━━━━━ 50s 10ms/step - custom_accuracy: 0.0550 - loss: 0.3899 - val_custom_accuracy: 0.3942 - val_loss: 0.2109
Epoch 2/1000
4688/4688 ━━━━━━━━━━━━━━━━━━━━ 46s 10ms/step - custom_accuracy: 0.2436 - loss: 0.2740 - val_custom_accuracy: 0.4798 - val_loss: 0.1733
Epoch 3/1000
4688/4688 ━━━━━━━━━━━━━━━━━━━━ 46s 10ms/step - custom_accuracy: 0.3112 - loss: 0.2440 - val_custom_accuracy: 0.5223 - val_loss: 0.1532
Epoch 4/1000
4688/4688 ━━━━━━━━━━━━━━━━━━━━ 46s 10ms/step - custom_accuracy: 0.3511 - loss: 0.2268 - val_custom_accuracy: 0.5546 - val_loss: 0.1410
Epoch 5/1000
4688/4688 ━━━━━━━━━━━━━━━━━━━━ 46s 10ms/step - custom_accuracy: 0.3796 - loss: 0.2140 - val_custom_accuracy: 0.5689 - val_loss: 0.1335
Epoch 5: early stopping
Restoring model weights from the end of the best epoch: 1.
